In [ ]:
''' want to formalize the process of creating these ENSP only files '''

In [5]:
import pandas as pd

In [15]:
# keep only the lines that contain 'ENSP'
readpath = '../funco_bench_raw/ash_father_benchmark.vcf'
writepath = '../funco_bench_ensp_tags/ash_father_ensp.vcf'

with open(readpath, 'r') as rp:
    with open(writepath, 'w') as wp:
        lines = rp.readlines()
        for line in lines:
            if 'ENSP' in line:
                wp.write(line)
                
    wp.close()
rp.close()    

In [16]:
# filter out the variant features we're not picking up 
readpath = '../funco_bench_ensp_tags/ash_father_ensp.vcf'
writepath = '../funco_bench_ensp_tags/ash_father_ensp_clean.vcf'

deplorables = ['|IGR|', '|FIVE_PRIME_UTR|', '|THREE_PRIME_UTR|', '|FIVE_PRIME_FLANK|', '|THREE_PRIME_FLANK|',
              '|COULD_NOT_DETERMINE|', '|SILENT|'] # does cerebra keep silent? 
keep_lines = 0
not_keep_lines = 0

with open(readpath, 'r') as rp:
    with open(writepath, 'w') as wp:
        lines = rp.readlines()
        for line in lines:
            if any(ele in line for ele in deplorables):
                not_keep_lines += 1
            else:
                keep_lines += 1
                wp.write(line)
                
    wp.close()
rp.close()    

In [36]:
# now lets pull out the ENSP tags
import os
readpath = '../funco_bench_ensp_tags/ash_father_ensp_clean.vcf'
writepath = '../funco_bench_ensp_tags/ash_father_ensp_only.vcf'

cmd = 'grep -E -o "ENSP.{0,20}" ' + readpath + ' > '+ writepath
os.system(cmd)

0

In [40]:
# and now get just the ENSP tags with '.p' in them
readpath = '../funco_bench_ensp_tags/ash_father_ensp_only.vcf'
writepath = '../funco_bench_ensp_tags/ash_father_ensp_filt.vcf'
cmd = "grep ':p.' " + readpath + ' > ' + writepath
os.system(cmd)

0

In [42]:
# and finally, keep only the ENSP trunk labels 
readpath = '../funco_bench_ensp_tags/ash_father_ensp_filt.vcf'
writepath = '../funco_bench_ensp_tags/ash_father_ensp_filt_final.vcf'

cmd = 'grep -E -o "ENSP.{0,11}" ' + readpath + ' > ' + writepath
os.system(cmd)

0